In [2]:
! pip install -q --upgrade google-generativeai langchain-google-genai python-dotenv

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import google.generativeai as genai
import os
genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

In [29]:
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [30]:
import json
with open("a song of ice and fire.json", "r", encoding="utf-8") as file:
    data_dict = json.load(file)

In [33]:

docs = ''.join(list(data_dict.values()))
# data = sum(docs.values())
# data

In [34]:
#documents = [{"title": i, "content": }]

In [35]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = text_splitter.create_documents([docs])
len(texts)
texts[:5]

[Document(page_content='丹妮莉丝·坦格利安（Daenerys Targaryen），又称丹妮或者“风暴降生”丹妮莉丝。她是“疯王”伊里斯·坦格利安二世之女，雷加·坦格利安王子的妹妹，也是古老的坦格利安王朝最后的传人。她在童年时与哥哥韦赛里斯·坦格利安一直被作为他人的政治筹码。韦赛里斯为了获取多斯拉克人的大军，将丹妮嫁给卓戈卡奥。自此丹妮莉丝逐渐成长为一个令人敬畏的女王和解放者。因为她从龙蛋中孵化出了三只幼龙，又被成为“龙之母”。丹妮莉丝是小说中一个主要的POV人物。在电视剧权力的游戏中，丹妮莉丝由演员Emilia'),
 Document(page_content='Clarke饰演。[1]丹妮莉丝身材娇小且美貌绝伦。她有着银金色的头发和紫色的眼睛。[2]幼年的丹妮莉丝是一个胆小而且不自信的小女孩，她的生活中只有流亡，而且非常依赖自己的哥哥韦赛里斯，而后者是一个性格残酷，语言粗暴，容易发怒，滥用暴力的人。她与卓戈卡奥的婚姻是她生命中的转折点，从那以后她逐渐成长为一个坚强且自信的女人。丹妮莉丝不认同韦赛里斯的懦弱和冷酷，但她和她的二哥却有一个相同的信念，那就是有朝一日重新君临维斯特洛，坐上铁王座。她认为这不仅是她天生的权力，也是她的义务和责任。自始至终，她都将这一点作为她的终极目标。其实她从未踏足维斯特落大陆，对那里的了解仅限于身边的乔拉·莫尔蒙和巴利斯坦'),
 Document(page_content='·赛尔弥等人的描述。在她的梦中，她更常看见布拉佛斯的红色大门，那里才是她的童年。她从未遗忘自己凄惨的童年，这使得她与其他潜在的征服者相比更具同情心。在她的统治时期，她将她所认为的“正义”和“公平”的执政方针贯穿始终。这集中表现在了她取缔奴隶制时的种种作为上。不过，虽然她的同情心和道德感在小说中十分鲜明，她在对待自己的敌人时却能够做到冷血无情。维斯特落流传一种说法：坦格利安不是疯狂就是伟大。丹妮莉丝似乎继承了坦格利安中那种天生的伟大领导力，或者叫吸引力。她的诸多追随者都对她死心塌地，又敬又爱。她常被拿来与她另一位十分具有人格魅力的哥哥雷加做类比。雷加的技巧、决心、才学和正义感都是他备受推崇的原因'),
 Document(page_content='。她作为统治者的主要弱点在于过于年轻，缺少统治经验。幸好她很聪明，能够很快从错误中学到新的智

In [36]:
#simple test
test_text = "丹妮莉丝·坦格利安（Daenerys Targaryen），又称丹妮或者“风暴降生”丹妮莉丝。她是“疯王”伊里斯·坦格利安二世之女，雷加·坦格利安王子的妹妹，也是古老的坦格利安王朝最后的传人。她在童年时与哥哥韦赛里斯·坦格利安一直被作为他人的政治筹码。韦赛里斯为了获取多斯拉克人的大军，将丹妮嫁给卓戈卡奥。自此丹妮莉丝逐渐成长为一个令人敬畏的女王和解放者。因为她从龙蛋中孵化出了三只幼龙，又被成为“龙之母”。丹妮莉丝是小说中一个主要的POV人物。在电视剧权力的游戏中，丹妮莉丝由演员Emilia"


In [57]:
query_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
doc_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", task_type="retrieval_document"
)
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(texts, embedding_function)
#test_text = "丹妮莉丝·坦格利安（Daenerys Targaryen），又称丹妮或者“风暴降生”丹妮莉丝。她是“疯王”伊里斯·坦格利安二世之女，雷加·坦格利安王子的妹妹，也是古老的坦格利安王朝最后的传人。她在童年时与哥哥韦赛里斯·坦格利安一直被作为他人的政治筹码。韦赛里斯为了获取多斯拉克人的大军，将丹妮嫁给卓戈卡奥。自此丹妮莉丝逐渐成长为一个令人敬畏的女王和解放者。因为她从龙蛋中孵化出了三只幼龙，又被成为“龙之母”。丹妮莉丝是小说中一个主要的POV人物。在电视剧权力的游戏中，丹妮莉丝由演员Emilia"
#query_embeddings.embed_documents(texts)


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [56]:

query = "石头"
query_embeddings.embed_query(query)
retriever = db.as_retriever()
retriever.get("tyrion lannister")

[Document(page_content='Lannister）爵士是泰温·兰尼斯特公爵的长弟，瑟曦、詹姆与提利昂的叔叔。他长期是泰温的得力助手，兄长死后成为家族的重要支柱。在电视剧中由Ian'),
 Document(page_content='Lannister）爵士是泰温·兰尼斯特公爵的长弟，瑟曦、詹姆与提利昂的叔叔。他长期是泰温的得力助手，兄长死后成为家族的重要支柱。在电视剧中由Ian'),
 Document(page_content='Lannister）爵士是泰温·兰尼斯特公爵的长弟，瑟曦、詹姆与提利昂的叔叔。他长期是泰温的得力助手，兄长死后成为家族的重要支柱。在电视剧中由Ian'),
 Document(page_content='Lannister）是泰温公爵和乔安娜夫人的长女，是詹姆·兰尼斯特的孪生姐姐。在篡夺者战争之后她嫁给了新王劳勃·拜拉席恩，成为七大王国的王后。在电视剧中由Lena')]

In [48]:
def make_qa_chain():
    llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)
    return RetrievalQA.from_chain_type(
        llm,
        retriever=db.as_retriever(),
        return_source_documents=True
    )
qa_chain = make_qa_chain()

In [49]:
q = "北境的领主有哪些？"
result = qa_chain({"query": q})
print(result["result"],"\nsources:",)
for idx, elt in enumerate(result["source_documents"]):
    print(elt.page_content)

The provided context does not mention anything about the North and its lords, so I cannot answer this question. 
sources:
亮后石头恢复了寂静。
坦格利安就是这些龙王家族之一
亮后石头恢复了寂静。
坦格利安就是这些龙王家族之一
